In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Data Analysis with Pyspark').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/26 11:30:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Agenta
We have google playstore dataset containing information of different apps installed rating and versions and other details and we are going to do analysis based on the data we have

1. find out top 10 reviews given to the apps
2. top 10 install apps and distribution of type (free/paid)
3. category wise distribution of installed apps
4. top paid apps
5. top paid rating apps

In [3]:
data = spark.read.csv(path='data/googlestore.csv', header=True, inferSchema=True, sep=';')

In [4]:
data.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+--------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|        Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+--------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4,1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|  Ιανουαρίου 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3,9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...| Ιανουαρίου 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite â...|ART_AND_DESIGN|   4,7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|   Αυγούστου 1, 2018|             

In [5]:
data.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [6]:
data.count()

10841

In [7]:
data = data.drop('Size', 'Content Rating', 'Last Updated', 'Android Ver')

In [8]:
from pyspark.sql.functions import col, regexp_replace

In [9]:
from pyspark.sql.types import IntegerType, FloatType, StringType

In [10]:
data = data\
.withColumn('Reviews', col('Reviews').cast(FloatType()))\
.withColumn('Installs', regexp_replace(col('Installs'), '[^0-9]', ''))\
.withColumn('Installs', col('Installs').cast(IntegerType()))\
.withColumn('Price', regexp_replace(col('Price'), '[$]', ''))\
.withColumn('Price', col('Price').cast(FloatType()))

In [11]:
data.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: float (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Current Ver: string (nullable = true)



In [50]:
# top reviewed apps
data.select('App', 'Reviews').groupBy('App').sum().orderBy('sum(Reviews)', ascending=False).show(10)

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|2.66241988E8|
|  WhatsApp Messenger|2.07348296E8|
|      Clash of Clans|1.79558784E8|
|Messenger â€“ Tex...|1.69932272E8|
|      Subway Surfers|1.66331958E8|
|    Candy Crush Saga|1.56993136E8|
|            Facebook|1.56286512E8|
|         8 Ball Pool| 9.9386198E7|
|        Clash Royale|   9.25303E7|
|            Snapchat|  6.804501E7|
+--------------------+------------+
only showing top 10 rows



In [61]:
from pyspark.sql.functions import sum

In [67]:
# app categories distribution
data.select('Category', 'Installs')\
.groupBy('Category').agg(sum('Installs').alias('Total Installs'))\
.orderBy('Total Installs', ascending=False).show()

+-------------------+--------------+
|           Category|Total Installs|
+-------------------+--------------+
|               GAME|   35086024415|
|      COMMUNICATION|   32647276251|
|       PRODUCTIVITY|   14176091369|
|             SOCIAL|   14069867902|
|              TOOLS|   11452771915|
|             FAMILY|   10258263505|
|        PHOTOGRAPHY|   10088247655|
| NEWS_AND_MAGAZINES|    7496317760|
|   TRAVEL_AND_LOCAL|    6868887146|
|      VIDEO_PLAYERS|    6222002720|
|           SHOPPING|    3247848785|
|      ENTERTAINMENT|    2869160000|
|    PERSONALIZATION|    2325494782|
|BOOKS_AND_REFERENCE|    1921469576|
|             SPORTS|    1751174498|
| HEALTH_AND_FITNESS|    1583072512|
|           BUSINESS|    1001914865|
|            FINANCE|     876648734|
|          EDUCATION|     871452000|
|MAPS_AND_NAVIGATION|     724281890|
+-------------------+--------------+
only showing top 20 rows



In [95]:
# top apps and their type
data.select('App', 'Installs', 'Type')\
.groupBy('App', 'Type').agg(sum('Installs').alias('Total Installs'))\
.orderBy('Total Installs', ascending=False).show()

+--------------------+----+--------------+
|                 App|Type|Total Installs|
+--------------------+----+--------------+
|      Subway Surfers|Free|    6000000000|
|         Google News|Free|    4000000000|
|        Google Drive|Free|    4000000000|
|           Instagram|Free|    4000000000|
|            Hangouts|Free|    4000000000|
|       Google Photos|Free|    4000000000|
|    Candy Crush Saga|Free|    3500000000|
|Messenger â€“ Tex...|Free|    3000000000|
|Skype - free IM &...|Free|    3000000000|
|        Temple Run 2|Free|    3000000000|
|  WhatsApp Messenger|Free|    3000000000|
|               Gmail|Free|    3000000000|
|Maps - Navigate &...|Free|    3000000000|
|Google Chrome: Fa...|Free|    3000000000|
|     Viber Messenger|Free|    2500000000|
|            Snapchat|Free|    2000000000|
|  Google Street View|Free|    2000000000|
|                 Pou|Free|    2000000000|
|             YouTube|Free|    2000000000|
|            Facebook|Free|    2000000000|
+----------

In [91]:
# top paid apps
data.select('App', 'Installs').filter(col('Type') == 'Paid').orderBy('Installs', ascending=False).show()

+--------------------+--------+
|                 App|Installs|
+--------------------+--------+
|           Minecraft|10000000|
|       Hitman Sniper|10000000|
|           Minecraft|10000000|
|Card Wars - Adven...| 1000000|
| Facetune - For Free| 1000000|
| Facetune - For Free| 1000000|
|Beautiful Widgets...| 1000000|
|   Cut the Rope GOLD| 1000000|
|          True Skate| 1000000|
| Facetune - For Free| 1000000|
|Beautiful Widgets...| 1000000|
|          HD Widgets| 1000000|
| DraStic DS Emulator| 1000000|
|Zombie Avengers:(...| 1000000|
|Sleep as Android ...| 1000000|
|Five Nights at Fr...| 1000000|
|Infinity Dungeon VIP| 1000000|
|          HD Widgets| 1000000|
|             Threema| 1000000|
|Grand Theft Auto:...| 1000000|
+--------------------+--------+
only showing top 20 rows



In [100]:
# top paid rating apps
data.select('App', 'Installs', 'Rating').filter(col('Type') == 'Paid').groupBy('App').sum('Installs').orderBy(['sum(Installs)'], ascending=False).show()

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|           Minecraft|     20000000|
|       Hitman Sniper|     10000000|
| Facetune - For Free|      3000000|
|          HD Widgets|      2000000|
|Beautiful Widgets...|      2000000|
|Infinity Dungeon VIP|      1000000|
| DraStic DS Emulator|      1000000|
| Fruit Ninja Classic|      1000000|
|             Threema|      1000000|
|          True Skate|      1000000|
|              Tasker|      1000000|
|Five Nights at Fr...|      1000000|
|   Cut the Rope GOLD|      1000000|
|Lost Journey (Dre...|      1000000|
|Card Wars - Adven...|      1000000|
|League of Stickma...|      1000000|
|     Toca Life: City|      1000000|
|   Where's My Water?|      1000000|
|Stickman Legends:...|      1000000|
|         Bloons TD 5|      1000000|
+--------------------+-------------+
only showing top 20 rows

